# 新一代Boost框架的使用

在[2.10 提升方法](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/杭电机器学习课程及代码/2.10%20提升方法.ipynb)中我们简单提到过，在SKLearn之外还有很多优秀的，基于Boosting Tree方法的衍生框架。包括XGBoost，LightGBM等，而各种开源比赛方案对它们的应用，也从事实上证明了这些框架的优秀性能。

因此这一节会简要介绍一些新框架的原理和用法。

参考文章：

- [对GBDT, Xgboost 和 LightGbm 的理解](https://zhuanlan.zhihu.com/p/60571936)

## XGBoost

### 简介

**XGBoost** 全称为“eXtreme Gradient Boosting”，是一种基于决策树的集成机器学习算法，使用梯度提升框架，适用于分类和回归问题。XGBoost 在各种比赛及比赛平台中大放异彩，比如Kaggle，天池，DataCastle等等，可以说是所有选手的必备武器之一。XGBoost 的项目主页参见[这里](https://xgboost.ai)。

XGBoost 最初由陈天奇开发。陈天奇本科毕业于上海交通大学 ACM 班，博士毕业于华盛顿大学计算机系，研究方向为大规模机器学习，2020年他将加入 CMU 出任助理教授。有兴趣的可以看看他的文章[《陈天奇：机器学习科研的十年》](http://www.sohu.com/a/328234576_129720)以及[访谈](https://cosx.org/2015/06/interview-of-tianqi)，也可以看看他的[知乎主页](https://www.zhihu.com/people/crowowrk/activities)。

下图展示了从决策树到 XGBoost 算法的发展过程：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/xgboost%E5%8F%91%E5%B1%95%E5%8E%86%E7%A8%8B.jpeg?raw=true)

图片刷不出来的请看[这里](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/xgboost发展历程.jpeg)

### 算法原理简述

我们从上一小节的图片中得知，XGBoost 是由 **GBDT** 发展而来。

首先回顾一下之前的知识：

**Boosting**方法，它假设我们的分类器是一系列基本分类器的线性组合：

$$\large f(x)=\sum^M_{m=1}\alpha_mG_m(x)$$

这里 $\alpha_m$ 就是弱分类器的系数，而 $G_m(x)$ 就是基本分类器。

而**提升树模型**就可以表示为决策树的加法模型：

$$\large f_M(x)=\sum_{m=1}^MT(x;\Theta)$$

其中，$T(x_i;\Theta_m)$ 表示第 $m$ 棵决策树， $\Theta_m$ 为决策树的参数，$M$ 为树的个数。

提升树算法采用**前向分步算法**。首先确定初始提升树 $f_0(x)=0$ ，这样一步一步迭代计算，第 $m$ 步的模型是

$$\large f_m(x)=f_{m-1}(x)+T(x;\Theta_m)$$